In [16]:
import pandas as pd
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import streamlit as st
import os

In [33]:
import langchain
import faiss
print("Langchain version:", langchain.__version__)
print("Faiss version:", faiss.__version__)

Langchain version: 0.2.17
Faiss version: 1.9.0


In [17]:
import pandas as pd

df = pd.read_csv('../final.csv')
df.head()

,회사이름,공고이름,url,경력,지역,직무
0,㈜인라이플,[주4.5일근무] 데이터 분석 및 기획자 채용,https://www.jobkorea.co.kr/Recruit/GI_Read/457...,경력2년↑,서울,데이터 분석
1,패스트파이브㈜,데이터 분석 담당자,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력5년↑,서울,데이터 분석
2,주식회사 빗썸,[빗썸] 비즈니스 데이터 분석,https://www.jobkorea.co.kr/Recruit/GI_Read/459...,경력7년↑,서울,데이터 분석
3,㈜컬리,[컬리] 온사이트 프로모션 데이터 분석,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력,서울,데이터 분석
4,"㈜숲(SOOP CO., LTD.)",[soop] 데이터 분석 및 지표 설계,https://www.jobkorea.co.kr/Recruit/GI_Read/460...,경력3년↑,경기,데이터 분석


In [18]:
# 데이터 로드
df = pd.read_csv('../final.csv')
df['SearchText'] = (
    "회사이름: " + df['회사이름'] + ", "
    "공고이름: " + df['공고이름'] + ", "
    "URL: " + df['url'] + ", "
    "경력: " + df['경력'] + ", "
    "지역: " + df['지역'] + ", "
    "직무: " + df['직무']
)

In [19]:
df['SearchText'].head()

0    회사이름: ㈜인라이플, 공고이름: [주4.5일근무] 데이터 분석 및 기획자 채용, ...
1    회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, URL: https://...
2    회사이름: 주식회사 빗썸, 공고이름: [빗썸] 비즈니스 데이터 분석, URL: ht...
3    회사이름: ㈜컬리, 공고이름: [컬리] 온사이트 프로모션 데이터 분석, URL: h...
4    회사이름: ㈜숲(SOOP CO., LTD.), 공고이름: [soop] 데이터 분석 ...
Name: SearchText, dtype: object

In [20]:
# 문서 생성
docs = [{"id": str(i), "text": row["SearchText"]} for i, row in df.iterrows()]
docs[:5]

[{'id': '0',
  'text': '회사이름: ㈜인라이플, 공고이름: [주4.5일근무] 데이터 분석 및 기획자 채용, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45711948?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=1, 경력: 경력2년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '1',
  'text': '회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46017897?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=2, 경력: 경력5년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '2',
  'text': '회사이름: 주식회사 빗썸, 공고이름: [빗썸] 비즈니스 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45906918?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=3, 경력: 경력7년↑, 지역: 서울, 직무: 데이터 분석'},
 {'id': '3',
  'text': '회사이름: ㈜컬리, 공고이름: [컬리] 온사이트 프로모션 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46002356?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=4, 경력: 경력, 지역: 서울, 직무: 데이터 분석'},
 {'id': '4',
  'text': '회사이름: ㈜숲(SOOP CO.

In [21]:
documents = [Document(page_content=doc["text"], metadata={"id": doc["id"]}) for doc in docs]
documents[:5]

[Document(metadata={'id': '0'}, page_content='회사이름: ㈜인라이플, 공고이름: [주4.5일근무] 데이터 분석 및 기획자 채용, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45711948?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=1, 경력: 경력2년↑, 지역: 서울, 직무: 데이터 분석'),
 Document(metadata={'id': '1'}, page_content='회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46017897?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=2, 경력: 경력5년↑, 지역: 서울, 직무: 데이터 분석'),
 Document(metadata={'id': '2'}, page_content='회사이름: 주식회사 빗썸, 공고이름: [빗썸] 비즈니스 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45906918?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=3, 경력: 경력7년↑, 지역: 서울, 직무: 데이터 분석'),
 Document(metadata={'id': '3'}, page_content='회사이름: ㈜컬리, 공고이름: [컬리] 온사이트 프로모션 데이터 분석, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46002356?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC

In [22]:
documents[-5:]

[Document(metadata={'id': '2360'}, page_content='회사이름: 그로잉랩, 공고이름: 웹 백엔드 개발(7년이상), URL: https://www.wanted.co.kr/wd/255209, 경력: 경력 7년 이상, 지역: 서울, 직무: API 개발자'),
 Document(metadata={'id': '2361'}, page_content='회사이름: 제로원에이아이, 공고이름: AI Back-End / Bigdata 엔지니어, URL: https://www.wanted.co.kr/wd/166683, 경력: 경력 3-10년, 지역: 경기, 직무: AI 서비스 개발자'),
 Document(metadata={'id': '2362'}, page_content='회사이름: 파이오링크, 공고이름: JAVA REACT / WEBFORNT-K GUI개발, URL: https://www.wanted.co.kr/wd/75911, 경력: 경력 1-10년, 지역: 서울, 직무: AI 서비스 개발자'),
 Document(metadata={'id': '2363'}, page_content='회사이름: 하이퍼리즘, 공고이름: Data Engineer, URL: https://www.wanted.co.kr/wd/229693, 경력: 신입-경력 15년, 지역: 서울, 직무: 데이터 엔지니어'),
 Document(metadata={'id': '2364'}, page_content='회사이름: 비모소프트, 공고이름: [인턴] 백엔드 개발자, URL: https://www.wanted.co.kr/wd/250137, 경력: 신입, 지역: 경기, 직무: API 개발자')]

In [23]:
# .env 파일 불러오기
load_dotenv()

# API 키 설정
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    st.error("API 키가 .env 파일에 설정되어 있지 않습니다.")
    st.stop()

os.environ["OPENAI_API_KEY"] = openai_api_key

In [24]:
# Embedding 생성 및 FAISS 인덱스 구축
try:
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)
except Exception as e:
    print(f"FAISS 구축 중 오류: {e}")
    exit()

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_21296\2361358348.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


In [25]:
# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [26]:
# 질문
question = "서울에 데이터 분석가의 채용공고를 알려줘"

In [ ]:
# 검색 및 Context 생성
retrieved_docs = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content for doc in retrieved_docs])

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_21296\2728549039.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(question)


In [ ]:
# Prompt 정의
prompt = PromptTemplate.from_template(
    """
    너는 주어진 구인공고 질문에 대답하는 AI야. 아래 제공된 context를 바탕으로 question에 정확히 대답해줘.
    최대 5개의 관련 구인공고를 다음 형식으로 제공해:
    - 회사이름: , 공고이름: , 지역: , URL: 

    만약 관련 데이터가 없다면 '관련 채용공고를 찾을 수 없습니다.'라고 대답해.
    
    # Context:
    {context}

    # Question:
    {question}

    # Answer:
    """
)

In [37]:
# LLM 및 Chain 구성
llm = ChatOpenAI(model="gpt-4", temperature=0.1)
chain = LLMChain(llm=llm, prompt=prompt)

# 답변 생성
if not retrieved_docs:
    print("관련 채용공고를 찾을 수 없습니다.")
else:
    result = chain.invoke({"context": context, "question": question})
    print(result)

{'context': '회사이름: ㈜에이브랩스, 공고이름: 데이터 분석 (data scientist) 채용, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46057474?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=41, 경력: 신입·경력, 지역: 서울, 직무: 데이터 분석\n회사이름: 씨앤토트㈜, 공고이름: 데이터 분석 및 시각화 (경력), URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46025939?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=15, 경력: 경력3년↑, 지역: 서울, 직무: 데이터 분석\n회사이름: 패스트파이브㈜, 공고이름: 데이터 분석 담당자, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/46017897?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=2, 경력: 경력5년↑, 지역: 서울, 직무: 데이터 분석\n회사이름: ㈜해양정보기술, 공고이름: 해양빅데이터 분석 및 해양수치모델링 부문 정규직 채용, URL: https://www.jobkorea.co.kr/Recruit/GI_Read/45913695?Oem_Code=C1&logpath=1&stext=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D&listno=47, 경력: 경력무관, 지역: 서울, 직무: 데이터 분석\n회사이름: 씨앤토트㈜, 공고이름: 빅데이터 분석 시스템 모델러, 데이터 아키텍트 (da) 모집 (경력), URL: https://www.jobkorea.co.kr/Recruit/GI_Read/4600310